# Tutorial 2: **Notebook** - *FVF*

> 🚧 **Under Construction**

**By 
gLayout Team**

__Content creators:__ Subham Pal, Saptarshi Ghosh

__Content reviewers:__ Mehedi Saligne

___
# Tutorial Objectives

This notebook is a tutorial on-

- **Importing** and **Placement** of FETs and other macros/Pcells with relative coordinates + Placing (and connecting) Via_stack on the ports of the FETs + encircling them with padrings.
- **Routing** between the placed *Vias* with *C_*, *L_*, *Straight_* and *smart* Routes and explaining the differences between these strategies. Particularly explaning the East/West/North/South angles, (For Example, why C has to parallel but L has to be perpendicular sides) and *Placing and connecting PINs for future LVS runs

## **Target** **Block** : **Flipped Voltage Follwer Cell**

The Flipped Voltage Follower (FVF) is a widely used analog building block known for its low output impedance, fast transient response, and ability to deliver large output current while maintaining voltage buffering properties. It is commonly employed in analog circuit design, particularly in output stages, gain boosting, common-mode feedback circuits, and super class-AB OTAs. 

(a) Conventional Voltage follower (common Drain); (b) Flipped voltage follower (FVF).

![](_images/FVF.png)

```bibtex
Domala, N., Sasikala, G. Low power flipped voltage follower current mirror with improved input output impedances. Sādhanā 46, 142 (2021). https://doi.org/10.1007/s12046-021-01665-6
```

## **NetList generation and LVS**
let's go through the step by step procedure to generate LVS and DRC clean layout of a FVF cell.

In [1]:
import os
import subprocess

# Run a shell, source .bashrc, then printenv
cmd = 'bash -c "source ~/.bashrc && printenv"'
result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
env_vars = {}
for line in result.stdout.splitlines():
    if '=' in line:
        key, value = line.split('=', 1)
        env_vars[key] = value

# Now, update os.environ with these
os.environ.update(env_vars)

In [2]:
from glayout import MappedPDK, sky130 , gf180
#from gdsfactory.cell import cell
from gdsfactory import Component
from gdsfactory.components import text_freetype, rectangle

In [3]:
from glayout import nmos, pmos
from glayout import via_stack
from glayout import rename_ports_by_orientation
from glayout import tapring

In [4]:
from glayout.util.comp_utils import evaluate_bbox, prec_center, prec_ref_center, align_comp_to_port
from glayout.util.port_utils import add_ports_perimeter,print_ports
from glayout.util.snap_to_grid import component_snap_to_grid
from glayout.spice.netlist import Netlist

In [5]:
from glayout.routing.straight_route import straight_route
from glayout.routing.c_route import c_route
from glayout.routing.L_route import L_route

FVF has two fets as shown in the schematic. We call M1 as input fet and M2 as feedback fet. Lets define arguments for the FETs

### 2. Basic Usage of the GLayout Framework
Each generator is a Python function that takes a `MappedPDK` object as a parameter and generates a DRC clean layout for the given PDK. The generator may also accept a set of optional layout parameters such as the width or length of a MOSFET. All parameters are normal Python function arguments.

The generator returns a `GDSFactory.Component` object that can be written to a `.gds` file and viewed using a tool such as Klayout. In this example, the `gdstk` library is used to convert the `.gds` file to an SVG image for viewing.

The pre-PEX SPICE netlist for the component can be viewed using `component.info['netlist'].generate_netlist()`.

In the following example the FET generator `glayout.primitives.fet` is imported and run with both the [Skywater 130](https://skywater-pdk.readthedocs.io/en/main/) and [GF180](https://gf180mcu-pdk.readthedocs.io/en/latest/) PDKs.

#### Demonstration of Basic Layout / Netlist Generation in SKY130 & GF180

In [6]:
from glayout import nmos,sky130,gf180

import gdstk
import svgutils.transform as sg
import IPython.display
from IPython.display import clear_output
import ipywidgets as widgets

# Used to display the results in a grid (notebook only)
left = widgets.Output()
leftSPICE = widgets.Output()
right = widgets.Output()
rightSPICE = widgets.Output()
hide = widgets.Output()

grid = widgets.GridspecLayout(1, 4)
grid[0, 0] = left
grid[0, 1] = leftSPICE
grid[0, 2] = right
grid[0, 3] = rightSPICE
display(grid)

def display_gds(gds_file, scale = 3):
  # Generate an SVG image
  top_level_cell = gdstk.read_gds(gds_file).top_level()[0]
  top_level_cell.write_svg('out.svg')

  # Scale the image for displaying
  fig = sg.fromfile('out.svg')
  fig.set_size((str(float(fig.width) * scale), str(float(fig.height) * scale)))
  fig.save('out.svg')

  # Display the image
  IPython.display.display(IPython.display.SVG('out.svg'))

def display_component(component, scale = 3):
  # Save to a GDS file
  with hide:
    component.write_gds("out.gds")

  display_gds('out.gds', scale)

with hide:
  # Generate the sky130 component
  component_sky130 = nmos(pdk = sky130, fingers=5)
  # Generate the gf180 component
  component_gf180 = nmos(pdk = gf180, fingers=5)

# Display the components' GDS and SPICE netlists
with left:
  print('Skywater 130nm N-MOSFET (fingers = 5)')
  display_component(component_sky130, scale=2)
with leftSPICE:
  print('Skywater 130nm SPICE Netlist')
  print(component_sky130.info['netlist'].generate_netlist())

with right:
  print('GF 180nm N-MOSFET (fingers = 5)')
  display_component(component_gf180, scale=2)
with rightSPICE:
  print('GF 180nm SPICE Netlist')
  print(component_gf180.info['netlist'].generate_netlist())

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [7]:
import sys
import os
sys.path.append(os.path.abspath("../../FVF"))

from my_FVF import flipped_voltage_follower,add_fvf_labels


In [8]:
comp = flipped_voltage_follower(gf180, device_type='nmos')
# comp.pprint_ports()
comp = add_fvf_labels(comp, gf180)
comp.name = "FVF"
#comp.write_gds('out_FVF.gds')
#comp.show()
#print("...Running DRC...")
drc_result = gf180.drc_magic(comp, "FVF")

2025-06-25 23:59:07.189 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmp_x1xxqvl/FVF.gds'


using default pdk_root: /usr/bin/miniconda3/share/pdk/
Defaulting to stale magic_commands.tcl

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Loading "/tmp/tmp_x1xxqvl/magic_commands.tcl" from command line.
Library written using GDS-II Release 6.0
Library name: library
Reading "FVF".
[INFO]: Loading FVF

Loading D

In [ ]:
fet_ref = c.insts["fet_ref"]
fet_ref = c.insts["fet_ref"]

In [10]:
comp.references()

TypeError: 'list' object is not callable

In [10]:
for inst in comp.insts.values():
            print(inst.name)

AttributeError: 'Component' object has no attribute 'insts'

In [ ]:
def fvf_netlist(fet_1: Component, fet_2: Component) -> Netlist:

    netlist = Netlist(circuit_name='FLIPPED_VOLTAGE_FOLLOWER', nodes=['VIN', 'VBULK', 'VOUT', 'Ib'])
    
    netlist.connect_netlist(fet_1.info['netlist'], [('D', 'Ib'), ('G', 'VIN'), ('S', 'VOUT'), ('B', 'VBULK')])
    netlist.connect_netlist(fet_2.info['netlist'], [('D', 'VOUT'), ('G', 'Ib'), ('S', 'VBULK'), ('B', 'VBULK')])
    
    return netlist



component.info['netlist'] = fvf_netlist(fet_1, fet_2)

### Run LVS
Design Rule Check ensures that the physical layout of an integrated circuit adheres to the manufacturing constraints defined by the foundry, such as minimum spacing, width, and enclosure rules. `Magic` is the tool we use for DRC here.

## Extraction and Post-Pex Simulation